In [1]:
import numpy as np
import pandas as pd

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# configure pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
dpath = "/media/cglab/TOSHIBA EXT/convertSPSS/DORRY/1. DORRY W1/1. DORRY W1 COMPLETE - 1001 to 1155/Parent Survey/Individual Measures/"

In [3]:
food = pd.read_csv(dpath + 'DORRY WAVE 1 - Parent - Food Insecurity Scale (FIS).csv')
print(food.shape)
food.head()

(144, 15)


,ID,FIS1PW1,FIS2PW1,FIS3PW1,FIS4PW1,FIS5PW1,FIS6PW1,FIS1PW1_R,FIS2PW1_R,FIS4PW1_R,FIS3PW1_R,FIS5PW1_R,FIS6PW1_R,FISPW1_T,FISPW1_L
0,1001,Sometimes True,Sometimes True,Yes,Only 1 or 2 months,Yes,Yes,=often or sometimes true OR almost every month...,=often or sometimes true OR almost every month...,=never true OR only 1 or 2 months OR do not know,=yes,=yes,=yes,5.0,3.0
1,1002,Never True,Never True,No,NaN,No,No,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=no,=no,=no,0.0,1.0
2,1003,Never True,Never True,No,NaN,No,No,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=no,=no,=no,0.0,1.0
3,1004,Never True,Never True,No,NaN,No,No,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=no,=no,=no,0.0,1.0
4,1005,Never True,Never True,No,NaN,No,No,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=no,=no,=no,0.0,1.0


In [4]:
food.dtypes

ID             int64
FIS1PW1       object
FIS2PW1       object
FIS3PW1       object
FIS4PW1       object
FIS5PW1       object
FIS6PW1       object
FIS1PW1_R     object
FIS2PW1_R     object
FIS4PW1_R     object
FIS3PW1_R     object
FIS5PW1_R     object
FIS6PW1_R     object
FISPW1_T     float64
FISPW1_L     float64
dtype: object

### Load RSA data to get subs who have Cross wavelet data

In [9]:
# function to get RSA subs
def getRSASubs():
    # load rsa
    rsa = pd.read_csv('/home/cglab/projects/dorry/cross-wavlet/ydi_rsa_stress_clean.csv')
    # select only subjects from DORRY study
    drsa= rsa[rsa['study']=='dorry'].copy()
    # get unique subject numbers
    subs_rsa_dor = drsa['id'].unique()
    return subs_rsa_dor

In [10]:
rsa_subs = getRSASubs()
rsa_subs

array([1002, 1005, 1007, 1008, 1012, 1013, 1014, 1015, 1016, 1017, 1018,
       1020, 1021, 1022, 1024, 1026, 1028, 1030, 1031, 1036, 1037, 1039,
       1040, 1041, 1042, 1044, 1045, 1048, 1049, 1050, 1051, 1053, 1054,
       1055, 1056, 1057, 1058, 1059, 1060, 1061, 1064, 1066, 1067, 1068,
       1069, 1070, 1071, 1073, 1076, 1078, 1083, 1085, 1086, 1087, 1089,
       1090, 1091, 1092, 1096, 1100, 1105, 1108, 1110, 1115, 1117, 1118])

In [11]:
# select subs from food that are in rsa_subs (i.e., subs with RSA and thus cross wavelet data)
food = food[food['ID'].isin(rsa_subs)]
print(food.shape)
food.head()

(65, 15)


,ID,FIS1PW1,FIS2PW1,FIS3PW1,FIS4PW1,FIS5PW1,FIS6PW1,FIS1PW1_R,FIS2PW1_R,FIS4PW1_R,FIS3PW1_R,FIS5PW1_R,FIS6PW1_R,FISPW1_T,FISPW1_L
1,1002,Never True,Never True,No,NaN,No,No,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=no,=no,=no,0.0,1.0
4,1005,Never True,Never True,No,NaN,No,No,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=no,=no,=no,0.0,1.0
6,1007,Often True,Sometimes True,Yes,Some months but not every month,Yes,No,=often or sometimes true OR almost every month...,=often or sometimes true OR almost every month...,=often or sometimes true OR almost every month...,=yes,=yes,=no,5.0,3.0
7,1008,Sometimes True,Sometimes True,No,NaN,No,No,=often or sometimes true OR almost every month...,=often or sometimes true OR almost every month...,=never true OR only 1 or 2 months OR do not know,=no,=no,=no,2.0,2.0
11,1012,Never True,Never True,No,NaN,No,No,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=never true OR only 1 or 2 months OR do not know,=no,=no,=no,0.0,1.0


In [13]:
food['FISPW1_T'].isnull().sum()

0

### save to csv

In [16]:
outpath = '/home/cglab/projects/dorry/cross-wavlet/'
food.to_csv(outpath + 'dorry_w1_p_FoodIS_matched_CWT.csv', index=False)